## Adding and Removing Genesets with GeMS REST-API (R)

Import libraries for dealing with HTTP requests

In [1]:
library(httr)
library(jsonlite)

### 1. Adding Genesets

The URL endpoint for handling geneset imports: `\insert`

In [2]:
BASE_URL_1 <- 'http://biocomp.bas.roche.com:1234/api/insert'

There are three fields for the input JSON object:

| Fields    | Type              | Description                                                       |
|:---------:|:----------------- |:----------------------------------------------------------------- |
| `params`  | `Dict`            | The command-line arguments for GMTx file upload (see below)       |
| `headers` | `List[Str]`       | Tab-separated **header** of the GMTx file format                  |
| `parsed`  | `List[List[Str]]` | Tab and line-separated **content** of the GMTx file format        |

The applicable arguments for the `params` field are listed below:

| Args | Name         | Required | Examples                |
|:----:|:------------:|:--------:|:----------------------- |
| `gf` | Gene format  | O        | 0, 1, 2, 3              |
| `so` | Source       | O        | Roche, MSigDB...        |
| `ti` | Taxonomy ID  | O        | 9606, 10090...          |
| `us` | User         | O        | Public, badil...        |
| `st` | Subtype      | X        | C7, BP...               |
| `do` | Domain       | X        | pathway, cell marker... |

In [3]:
params <- list(gf = 0, so = 'Roche', ti = 9606, us = 'kanga6')

headers <- c('setName', 'desc', 'genes')

parsed <- list(
    c('TIROSH_exhaution_marker_subset', 'subset from TRIOSH table S14', 'CXCL13', 'TNFRSF1B', 'RGS2'), 
    c('Speiser_EBV_subset_DN', 'subset from Speiser_table S2', 'ACADSB', 'AFTPH', 'ARL1')
)

dataIn_1 <- list (
    headers = headers,
    parsed = parsed,
    params = params
)

The fields `headers` and `parsed`, in essence, contains a deconstructed GMTx file. If you have your genesets in a some other format, just write your own helper function. 

Here is an example with the same geneset as above (convert `genesets` to `parsed`):

In [4]:
genesets <- list(
    list(
        setName = 'TIROSH_exhaution_marker_subset', 
        desc = 'subset from TRIOSH table S14', 
        genes = c('CXCL13', 'TNFRSF1B', 'RGS2')
    ),
    list(
        setName = 'Speiser_EBV_subset_DN', 
        desc = 'subset from Speiser_table S2', 
        genes = c('ACADSB', 'AFTPH', 'ARL1')
    )
)

parsed <- list()
for (geneset in genesets){
    sParsed <- c()
    for (header in headers){
        sParsed <- c(sParsed, geneset[[header]])
    }
    parsed <- append(parsed, list(sParsed))
}

The payload is sent via an HTTP POST request, which returns the response as a JSON object. The first element of the `response` array should show either a 200 (good request) or a 404 (bad request). Any error messages are listed in the proceeding elements.

In [5]:
response <- POST(BASE_URL_1, body=dataIn_1, encode='json')
returnJSON <- fromJSON(httr::content(response, 'text'))
print(returnJSON)

No encoding supplied: defaulting to UTF-8.


$response
[1] 200



### 2. Removing Genesets

The URL endpoint for handling geneset removals: `\remove`

In [6]:
BASE_URL_2 <- 'http://biocomp:1234/api/remove'

There is one field for the input JSON object:

| Field     | Type         | Description                                                             |
|:---------:|:------------ |:----------------------------------------------------------------------- |
| `genesets`| `List[Dict]` | Array of JSON objects with keys: `setName`, `source`, `user`, `subtype` | 

In [7]:
toRemove <- list(
    list(
        setName = 'TIROSH_exhaution_marker_subset', 
        source = 'Roche', 
        user = 'kanga6', 
        subtype = ''
    ),
    list(
        setName = 'Speiser_EBV_subset_DN', 
        source = 'Roche', 
        user = 'kanga6', 
        subtype = ''
    )
)

dataIn_2 <- list(
    genesets = toRemove
)

The payload is sent via an HTTP POST request, which returns the response as a JSON object. The first element of the `response` array should show either a 200 (good request) or a 404 (bad request). Any error messages are listed in the proceeding elements.

In [8]:
response <- POST(BASE_URL_2, body=dataIn_2, encode='json')
returnJSON <- fromJSON(httr::content(response, 'text'))
print(returnJSON)

No encoding supplied: defaulting to UTF-8.


$response
[1] 200

